<a href="https://colab.research.google.com/github/andrehochuli/teaching/blob/main/ComputerVision/Lecture%2005%20-%20Feature%20Extraction/Lecture_05_Avalia%C3%A7%C3%A3o_Formativa_Lecture_NOTES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Auxiliary Functions

In [ ]:
import numpy as np
import cv2, math
import matplotlib.pyplot as plt
#Keras to import datasets, not for deep learning (yet)
from tensorflow import keras
from sklearn.decomposition import PCA
from sklearn import metrics, preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.manifold import TSNE
import seaborn as sns
import pandas as pd
import skimage.feature as feature
from sklearn.model_selection import train_test_split
import os
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

In [ ]:
#Auxiliary Function to plot side by side
#@Author: Prof. André Hochuli
#Visualiza um lista de figuras lado a lado, facilitando a comparação qualitativa
def plot_sidebyside(img_list,titles=None,colormap=None,figsize=(12,6)):
  n = len(img_list)
  figure, axis = plt.subplots(1, n, figsize=figsize)

  if titles is None:
    titles = []
    A = ord('A')
    for i in range(n):
      titles.append(chr(A+i))

  for i in range(n):
    axis[i].imshow(img_list[i], cmap=colormap)
    axis[i].set_title(titles[i])
    axis[i].axis('off')
  # Combine all the operations and display
  plt.show()

In [ ]:
#@Author: Prof. André Hochuli
#Compila os resultados para analises qualitativas e quantitativas
def performance_evaluation(x_test, y_test, predictions, class_names, info_message):

    print(f"Evaluation of {info_message}")
    print(metrics.classification_report(y_test, predictions))


    # Matriz de confusão
    disp = metrics.ConfusionMatrixDisplay.from_predictions(y_test, predictions)
    disp.figure_.suptitle("Confusion Matrix")
    plt.show()

    # Imagens classificadas corretamente
    correct_idx = np.where(y_test == predictions)[0]
    n_correct = min(10, len(correct_idx))
    if n_correct > 0:
        plt.figure(figsize=(22, 4))
        for i in range(n_correct):
            idx = correct_idx[i]
            plt.subplot(1, n_correct, i+1)
            plt.imshow(x_test[idx], cmap='gray', interpolation='nearest')
            plt.axis('off')
            plt.title(f"Lbl:{y_test[idx]} Pred:{predictions[idx]}")
        plt.suptitle("Correct Predictions", fontsize=16, fontweight='bold', color='white', backgroundcolor='green')
        plt.show()

    #Imagens classificadas incorretamente
    wrong_idx = np.where(y_test != predictions)[0]
    n_wrong = min(10, len(wrong_idx))
    if n_wrong > 0:
        plt.figure(figsize=(22, 4))
        for i in range(n_wrong):
            idx = wrong_idx[i]
            plt.subplot(1, n_wrong, i+1)
            plt.imshow(x_test[idx], cmap='gray', interpolation='nearest')
            plt.axis('off')
            plt.title(f"Lbl:{y_test[idx]} Pred:{predictions[idx]}")
        plt.suptitle("Wrong Predictions", fontsize=16, fontweight='bold', color='white', backgroundcolor='red')
        plt.show()


    #Exibir exemplo de cada classe
    unique_classes = np.unique(y_test)
    plt.figure(figsize=(22, 4))
    for i, cls in enumerate(unique_classes):
        idx = np.where(y_test == cls)[0][0]  # primeiro índice da classe
        plt.subplot(1, len(unique_classes), i+1)
        plt.imshow(x_test[idx], cmap='gray', interpolation='nearest')
        plt.axis('off')
        plt.title(f"{i}-{class_names[cls]}")
    plt.suptitle("Example of each class", fontsize=16, fontweight='bold', color='black', backgroundcolor='yellow')
    plt.show()


In [ ]:
def random_shuffle(X_train, y_train, X_test, y_test, random_state=42):
    np.random.seed(random_state)

    # Shuffle train
    idx_train = np.random.permutation(len(y_train))
    X_train_shuffled = X_train[idx_train]
    y_train_shuffled = y_train[idx_train]

    # Shuffle test
    idx_test = np.random.permutation(len(y_test))
    X_test_shuffled = X_test[idx_test]
    y_test_shuffled = y_test[idx_test]

    return X_train_shuffled, y_train_shuffled, X_test_shuffled, y_test_shuffled

In [ ]:
import numpy as np
from collections import Counter

def random_undersampling(X, y, random_state=42):

    np.random.seed(random_state)

    # Contar amostras por classe
    class_counts = Counter(y)
    min_count = min(class_counts.values())

    X_resampled = []
    y_resampled = []

    for cls in class_counts.keys():
        idx_cls = np.where(y == cls)[0]
        # Seleciona min_count índices aleatoriamente
        selected_idx = np.random.choice(idx_cls, size=min_count, replace=False)
        X_resampled.append(X[selected_idx])
        y_resampled.append(y[selected_idx])

    X_resampled = np.concatenate(X_resampled, axis=0)
    y_resampled = np.concatenate(y_resampled, axis=0)

    # Embaralhar os dados novamente
    shuffle_idx = np.random.permutation(len(y_resampled))
    X_resampled = X_resampled[shuffle_idx]
    y_resampled = y_resampled[shuffle_idx]

    return X_resampled, y_resampled


In [ ]:
def retrieve_swedish_leaf(base_dir):
    dataset_url = "http://www.ppgia.pucpr.br/~aghochuli/swedish_leaf.zip"
    zip_path = "swedish_leaf.zip"

    if not os.path.exists(base_dir):
      print("Downloading...")
      os.system(f"wget -O {zip_path} {dataset_url}")
      os.system(f"unzip {zip_path}")
      #os.remove(zip_path)
      print(f"Dataset extracted to: {base_dir}")
    else:
      print(f"Dataset already available at: {base_dir}")

def load_swedish_leaf():

    base_dir = "swedish_leaf"

    retrieve_swedish_leaf(base_dir)

    x_train = []
    y_train = []

    for i in range(1, 9):
        leaf_dir = os.path.join(base_dir, f'leaf{i}')
        if os.path.isdir(leaf_dir):

            for filename in os.listdir(leaf_dir):
                if filename.endswith('.png'):
                    img_path = os.path.join(leaf_dir, filename)
                    try:
                        # Carrega em escala de cinza
                        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                        # Redimensiona para 128x128
                        img_resized = cv2.resize(img, (128, 128))

                        x_train.append(img_resized)
                        y_train.append(i - 1)  # Labels de 0 a 7
                    except Exception as e:
                        print(f"Error loading image {img_path}: {e}")

    # Holdout 70/30 estratificado
    X_train, X_test, y_train_split, y_test_split = train_test_split(
        x_train, y_train,
        test_size=0.3,
        random_state=42,
        stratify=y_train
    )

    class_names = {
      0: 'Ulmus carpinifolia',
      1: 'Acer',
      2: 'Salix aurita',
      3: 'Quercus',
      4: 'Alnus incana',
      5: 'Betula pubescens',
      6: 'Salix alba \'Sericea\'',
      7: 'Populus tremula'
    }

    return np.array(X_train),np.array(y_train_split),np.array(X_test),np.array(y_test_split), class_names

In [ ]:
def retrieve_paper_rock_scissors(base_dir):
    dataset_url = "http://www.ppgia.pucpr.br/~aghochuli/paper-rock-scissors.zip"
    zip_path = "paper-rock-scissors.zip"

    if not os.path.exists(base_dir):
        print("Downloading...")
        os.system(f"wget -O {zip_path} {dataset_url}")
        os.system(f"unzip {zip_path}")
        #os.remove(zip_path)
        print(f"Dataset extracted to: {base_dir}")
    else:
        print(f"Dataset already available at: {base_dir}")

def load_paper_rock_scissors():

    base_dir = "paper-rock-scissors"

    retrieve_paper_rock_scissors(base_dir)

    x_data = []
    y_data = []

    # Mapeamento das classes
    class_names = {
        0: 'paper',
        1: 'rock',
        2: 'scissors'
    }

    for label, cls in class_names.items():
        cls_dir = os.path.join(base_dir, cls)
        if os.path.isdir(cls_dir):
            for filename in os.listdir(cls_dir):
                if filename.endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(cls_dir, filename)
                    try:
                        # Carrega em escala de cinza
                        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                        # Redimensiona para 128x128
                        img_resized = cv2.resize(img, (128, 128))
                        x_data.append(img_resized)
                        y_data.append(label)
                    except Exception as e:
                        print(f"Error loading image {img_path}: {e}")

    # Holdout 70/30 estratificado
    X_train, X_test, y_train_split, y_test_split = train_test_split(
        x_data, y_data,
        test_size=0.3,
        random_state=42,
        stratify=y_data
    )

    return (
        np.array(X_train),
        np.array(y_train_split),
        np.array(X_test),
        np.array(y_test_split),
        class_names
    )

In [ ]:
import os
import cv2
import numpy as np

def retrieve_vehicle(base_dir):
    dataset_url = "http://www.ppgia.pucpr.br/~aghochuli/vehicle.zip"
    zip_path = "vehicle.zip"

    if not os.path.exists(base_dir):
        print("Downloading...")
        os.system(f"wget -O {zip_path} {dataset_url}")
        os.system(f"unzip {zip_path}")
        #os.remove(zip_path)
        print(f"Dataset extracted to: {base_dir}")
    else:
        print(f"Dataset already available at: {base_dir}")

def load_vehicle():
    base_dir = "vehicle"
    retrieve_vehicle(base_dir)

    # Estrutura esperada
    subsets = ['train', 'test']
    class_names = {}
    data = {}

    for subset in subsets:
        subset_dir = os.path.join(base_dir, subset)
        x_data = []
        y_data = []
        if os.path.isdir(subset_dir):
            classes = [d for d in os.listdir(subset_dir) if os.path.isdir(os.path.join(subset_dir, d))]
            # Mapear classes para labels se ainda não mapeadas
            for idx, cls in enumerate(classes):
                if cls not in class_names.values():
                    class_names[idx] = cls
            # Carregar imagens
            for label, cls in class_names.items():
                cls_dir = os.path.join(subset_dir, cls)
                if os.path.isdir(cls_dir):
                    for filename in os.listdir(cls_dir):
                        if filename.endswith(('.png', '.jpg', '.jpeg')):
                            img_path = os.path.join(cls_dir, filename)
                            try:
                                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                                img_resized = cv2.resize(img, (128, 128))
                                x_data.append(img_resized)
                                y_data.append(label)
                            except Exception as e:
                                print(f"Error loading image {img_path}: {e}")
        data[subset] = (np.array(x_data), np.array(y_data))

    return data['train'][0], data['train'][1], data['test'][0], data['test'][1], class_names

# Descriptors

In [ ]:
class EdgeDescriptor:
    def __init__(self, method='canny', low_threshold=50, high_threshold=150):
        """
        Descritor baseado em bordas.
        method: 'canny' ou 'sobel'
        """
        self.method = method.lower()
        self.low_threshold = low_threshold
        self.high_threshold = high_threshold

    def describe(self, image, visualize=False):
        """
        Retorna um vetor de features baseado em bordas.
        Se visualize=True, retorna também a imagem de bordas.
        """
        # Converte para escala de cinza se necessário
        if len(image.shape) > 2 and image.shape[2] == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()

        # Aplicar detector de bordas
        if self.method == 'canny':
            edges = cv2.Canny(gray, self.low_threshold, self.high_threshold)
        elif self.method == 'sobel':
            sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
            sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
            edges = cv2.magnitude(sobelx, sobely)
            edges = np.uint8(edges)
        else:
            raise ValueError("method deve ser 'canny' ou 'sobel'")

        if visualize:
            return edges.flatten(), edges
        else:
            return edges.flatten()


In [ ]:
class ProjectionHistogramDescriptor:
    def __init__(self, normalize=True):
        """
        Descritor baseado em histogramas de projeção horizontal e vertical.
        normalize: se True, normaliza os histogramas
        """
        self.normalize = normalize

    def describe(self, image, visualize=False):
        """
        Retorna o vetor de features de projeção.
        Se visualize=True, retorna também uma figura com histogramas.
        """
        # Converte para escala de cinza se necessário
        if len(image.shape) > 2 and image.shape[2] == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()

        # Histogramas de projeção
        horizontal_proj = np.sum(gray, axis=1)
        vertical_proj = np.sum(gray, axis=0)

        if self.normalize:
            horizontal_proj = horizontal_proj / (horizontal_proj.sum() + 1e-7)
            vertical_proj = vertical_proj / (vertical_proj.sum() + 1e-7)

        features = np.concatenate([horizontal_proj, vertical_proj])

        if visualize:
            fig, ax = plt.subplots(1, 2, figsize=(10,3))
            ax[0].bar(range(len(horizontal_proj)), horizontal_proj, color='gray')
            ax[0].set_title("Horizontal Projection")
            ax[1].bar(range(len(vertical_proj)), vertical_proj, color='gray')
            ax[1].set_title("Vertical Projection")
            plt.tight_layout()
            plt.close(fig)  # não exibe automaticamente
            return features, fig
        else:
            return features

In [ ]:
class ColorDescriptor:
    def __init__(self, bins=(8, 8, 8), color_space='HSV'):
        """
        bins: número de bins por canal
        color_space: 'HSV' ou 'RGB'
        """
        self.bins = bins
        self.color_space = color_space.upper()
        if self.color_space not in ['HSV', 'RGB']:
            raise ValueError("color_space deve ser 'HSV' ou 'RGB'")

    def describe(self, image, mask=None, visualize=False):
        """
        Retorna o histograma de cor concatenado e normalizado.
        Se visualize=True, retorna também a imagem do histograma.
        """
        # Converte a imagem para o espaço de cor desejado
        if self.color_space == 'HSV':
            image_conv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV) if image.shape[2]==3 else image
            hist_range = [0, 180, 0, 256, 0, 256]
        else:  # RGB
            image_conv = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) if image.shape[2]==3 else image
            hist_range = [0, 256, 0, 256, 0, 256]

        # Calcula histograma 3D
        hist = cv2.calcHist([image_conv], [0, 1, 2], mask, self.bins, hist_range)
        hist = cv2.normalize(hist, hist).flatten()

        if visualize:
            # Criar uma imagem simples do histograma para inspeção
            fig, ax = plt.subplots(figsize=(5,2))
            ax.bar(range(len(hist)), hist, color='gray')
            ax.set_title(f"{self.color_space} Histogram")
            ax.set_xlabel("Bins")
            ax.set_ylabel("Normalized Count")
            ax.set_xticks([])
            plt.close(fig)  # não exibe automaticamente
            return hist, fig
        else:
            return hist

In [ ]:
class HuMomentsDescriptor:
    def describe(self, im, threshold_val=128, visualize=False):
        """
        Calcula o vetor de Hu Moments de uma imagem.
        - im: imagem RGB ou grayscale
        - threshold_val: valor para binarização
        - visualize: se True, retorna também a imagem binarizada
        """
        # Converte para grayscale se necessário
        if len(im.shape) > 2 and im.shape[2] == 3:
            im = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)

        # Binariza a imagem
        _, thresh = cv2.threshold(im, threshold_val, 255, cv2.THRESH_BINARY)

        # Calcula momentos e Hu Moments
        moments = cv2.moments(thresh)
        huMoments = cv2.HuMoments(moments)

        # Escala logarítmica
        for i in range(len(huMoments)):
            if huMoments[i] != 0:
                huMoments[i] = -1 * math.copysign(1.0, huMoments[i]) * math.log10(abs(huMoments[i]))

        hu_vector = huMoments.reshape(-1)

        if visualize:
            return hu_vector, thresh
        else:
            return hu_vector

In [ ]:
class HogDescriptor:
    def __init__(self, orientations=9, pixels_per_cell=(8,8), cells_per_block=(2,2), resize_dim=(64,128)):
        """
        Classe para extrair features HOG.
        """
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block
        self.resize_dim = resize_dim

    def describe(self, im, visualize=False):
        """
        Extrai o vetor de features HOG de uma imagem.
        - im: imagem RGB ou grayscale
        - visualize: se True, retorna também a imagem HOG
        """
        # Converte para grayscale se necessário
        if len(im.shape) > 2 and im.shape[2] == 3:
            im = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)

        # Redimensiona a imagem
        im = cv2.resize(im, self.resize_dim)

        # Extrai HOG
        fd, hog_image = feature.hog(im,
                                    orientations=self.orientations,
                                    pixels_per_cell=self.pixels_per_cell,
                                    cells_per_block=self.cells_per_block,
                                    block_norm='L2-Hys',
                                    visualize=True)
        if visualize:
            return fd, hog_image
        else:
            return fd

In [ ]:
class GaborDescriptor:
    def __init__(self, ksize=31, n_filters=8, sigma=4.0, lambd=10.0, gamma=0.5, psi=0):
        """
        Classe para extrair features Gabor compactas.
        Cada filtro gera estatísticas (média, variância, desvio padrão) para reduzir dimensionalidade.
        """
        self.filters = []
        for theta in np.arange(0, np.pi, np.pi / n_filters):
            kern = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, psi, ktype=cv2.CV_32F)
            kern /= (kern.sum() + 1e-7)  # normalização segura
            self.filters.append(kern)

    def get_filters(self):
        """Retorna a lista de filtros"""
        return self.filters

    def describe(self, im, visualize=False):
        """
        Retorna um vetor compacto de features Gabor baseado em estatísticas.
        Se visualize=True, retorna também a imagem filtrada do primeiro filtro.
        - im: imagem RGB ou escala de cinza
        """
        # Converte para escala de cinza se necessário
        if len(im.shape) > 2 and im.shape[2] == 3:
            im_gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
        else:
            im_gray = im.copy()

        im_gray = cv2.resize(im_gray, (128, 128)).astype(np.float32)

        feats = []
        first_filtered = None
        for i, kern in enumerate(self.filters):
            f_im = cv2.filter2D(im_gray, cv2.CV_32F, kern)
            feats.extend([f_im.mean(), f_im.var(), f_im.std()])
            if visualize and i == 0:
                first_filtered = f_im.copy()

        if visualize:
            return np.array(feats), first_filtered
        else:
            return np.array(feats)


In [ ]:
class LocalBinaryPatternsDescriptor:
    def __init__(self, numPoints, radius):
        """
        Descritor LBP (Local Binary Patterns).
        numPoints: número de vizinhos
        radius: raio
        """
        self.numPoints = numPoints
        self.radius = radius

    def describe(self, image, eps=1e-7, visualize=False):
        """
        Retorna o histograma normalizado LBP.
        Se visualize=True, retorna também a imagem LBP.
        """
        # Converte para grayscale se necessário
        if len(image.shape) > 2 and image.shape[2] == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        else:
            gray = image.copy()

        # Calcula LBP
        lbp_img = feature.local_binary_pattern(gray, self.numPoints, self.radius, method="uniform")
        hist, _ = np.histogram(lbp_img.ravel(),
                               bins=np.arange(0, self.numPoints + 3),
                               range=(0, self.numPoints + 2))
        # Normaliza
        hist = hist.astype('float')
        hist /= (hist.sum() + eps)

        if visualize:
            return hist, lbp_img
        else:
            return hist

# Load Data

In [ ]:
#x_train, y_train, x_test, y_test, class_names = load_swedish_leaf()
#x_train, y_train, x_test, y_test, class_names = load_paper_rock_scissors()
x_train, y_train, x_test, y_test, class_names = load_vehicle()




In [ ]:
#Embaralha para evitar viés de classe
x_train, y_train, x_test, y_test = random_shuffle(x_train, y_train, x_test, y_test)

In [ ]:
x_train, y_train = random_undersampling(x_train,y_train)

In [ ]:
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)

#y_all = np.concatenate([y_train, y_test])
y_all = y_train
unique_classes, counts = np.unique(y_all, return_counts=True)

plt.figure(figsize=(12, 6))
plt.bar(range(len(unique_classes)), counts, tick_label=[class_names[i] for i in unique_classes])
plt.xticks(rotation=90)
plt.xlabel("Classes")
plt.ylabel("Número de amostras")
plt.title("Distribuição das classes")
plt.tight_layout()
plt.show()

In [ ]:
samples_per_class = 5
for cls in np.unique(y_train):
    idxs = np.where(y_train == cls)[0]
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    imgs = [x_train[i] for i in idxs]
    titles = [f"{class_names[cls]} #{i+1}" for i in range(samples_per_class)]
    plot_sidebyside(imgs, titles=titles, colormap="gray")

In [ ]:
features_train = {}
features_test = {}

descriptors = [
    LocalBinaryPatternsDescriptor(8, 2),
    HogDescriptor(),
    Projecto(),
    GaborDescriptor(n_filters=8)
]

desc_names = ['LBP', 'HOG', 'HU', 'GABOR']

for desc, name in zip(descriptors, desc_names):
    print(f"Extraindo features {name}...")
    feat_tr = np.array([desc.describe(img) for img in x_train]).reshape(len(x_train), -1)
    feat_te = np.array([desc.describe(img) for img in x_test]).reshape(len(x_test), -1)
    features_train[name] = feat_tr
    features_test[name] = feat_te
    print(f"{name} -> train: {feat_tr.shape}, test: {feat_te.shape}")


In [ ]:
classifiers = {
    'SVM': SVC(kernel='linear', C=1.0),
    'RandomForest': RandomForestClassifier(n_estimators=100),
    'MLP': MLPClassifier(hidden_layer_sizes=(128,64), max_iter=300)
}

# Testar cada classificador
for clf_name, clf in classifiers.items():
    print(f"\n=== Classificador: {clf_name} ===")
    for desc_name in desc_names:
        print('#' * 35)
        print(f"###### Usando features: {desc_name} #####")
        print('#' * 35)

        X_tr = features_train[desc_name]
        X_te = features_test[desc_name]

        # Aplicar StandardScaler
        scaler = StandardScaler()
        X_tr_scaled = scaler.fit_transform(X_tr)
        X_te_scaled = scaler.transform(X_te)


        # Treinar
        clf.fit(X_tr_scaled, y_train)
        preds = clf.predict(X_te_scaled)

        # Avaliar usando a função performance_evaluation
        performance_evaluation(x_test, y_test, preds, class_names, f"{clf_name} com {desc_name}")



In [ ]:
# Combinar todos os descritores
X_train_combined = np.concatenate(list(features_train.values()), axis=1)
X_test_combined = np.concatenate(list(features_test.values()), axis=1)

print("Combinado -> train:", X_train_combined.shape, "test:", X_test_combined.shape)


# Normalização
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_combined)
X_test_scaled = scaler.transform(X_test_combined)


# Reduzir dimensionalidade com PCA
pca = PCA(n_components=0.97, random_state=42)  # mantém 97% da variância
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

print("Após PCA -> train:", X_train_pca.shape, "test:", X_test_pca.shape)

# Definição dos classificadores
classifiers = {
    'SVM': SVC(kernel='linear', C=1.0),
    'RandomForest': RandomForestClassifier(n_estimators=100),
    'MLP': MLPClassifier(hidden_layer_sizes=(128,64), max_iter=300)
}

# Treinamento e avaliação
for name, clf in classifiers.items():
    print(f"\nTreinando {name}...")
    clf.fit(X_train_pca, y_train)
    acc = clf.score(X_test_pca, y_test)
    print(f"{name} Accuracy: {acc:.4f}")